# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver = webdriver.Chrome()
url = 'https://jportal.mdcourts.gov/license/pbIndex.jsp?ac=y'
driver.get(url)

In [3]:
checkbox = driver.find_element_by_id("checkbox")
checkbox.click()

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [4]:
button = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
button.click()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [5]:
goto_search = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a')
goto_search.click()

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [6]:
select = Select(driver.find_element_by_name('slcJurisdiction'))
select.select_by_visible_text('Statewide')

In [7]:
text_input = driver.find_element_by_id('txtTradeName')
text_input.send_keys("VAP%")

In [8]:
text_input.submit()

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [9]:
search_list = driver.find_elements_by_class_name('searchlistitem')
for item in search_list:
    print(item.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [10]:
# search_list = driver.find_elements_by_class_name('searchfieldtitle')
# tds = search_list[0].find_elements_by_tag_name('td')
# click_for_details = tds[2].find_element_by_tag_name('a')
# click_for_details.click()

In [11]:
# I GOT THE STALE ERROR PRETTY EARLY IN THE PROCESS -- AFTER I'D ONLY BUILT THE LOOP TO GRAB
# THE MAILING ADDRESS AND THEN RETURN TO SEARCH RESULTS (WAS DOING ONE PIECE AT A TIME AND TESTING)

search_list = driver.find_elements_by_class_name('searchfieldtitle')

for item in search_list[0:]:
    tds = item.find_elements_by_tag_name('td')
    try:
        click_for_details = tds[2].find_element_by_tag_name('a')
        click_for_details.click()
        rows = driver.find_elements_by_class_name('tablecelltext')
        mailing_address = "\n".join(rows[0].find_elements_by_tag_name('td')[0].text.split('\n')[-2:])
        print(mailing_address)
        
        return_to_list_button = driver.find_element_by_partial_link_text('Retur')
        return_to_list_button.click()
        
    except:
        print(item.text, "Status pending")

1104 PLANTERS PLACE
SALISBURY, MD 21804


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.4 x86_64)


In [ ]:
# this is just my cell to test more simply whether my code for getting address worked before
# adding the the for loop
# rows = driver.find_elements_by_class_name('tablecelltext')
# len(rows)
# print("\n".join(rows[0].find_elements_by_tag_name('td')[0].text.split('\n')[-2:]))

# tables = driver.find_elements_by_tag_name('table')
# print(tables[-2].find_elements_by_tag_name('td')[0].text)

# return_to_list_button = driver.find_element_by_partial_link_text('Retur')
# driver.execute_script("arguments[0].scrollIntoView(true)", return_to_list_button)
# return_to_list_button.click()

# driver.find_elements_by_tag_name('table')[-2].find_elements_by_tag_name('td')[0].find_elements_by_tag_name('a')

Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [ ]:
search_list = driver.find_elements_by_class_name('searchfieldtitle')
list_for_df = []

for result in search_list:
    dict = {}
#    print(result.find_element_by_class_name('searchlistitem').text)
    tds = result.find_elements_by_tag_name('td')
    dict['name'] = tds[1].text
    try:
        dict['url'] = tds[2].find_element_by_tag_name('a').get_attribute('href')
    except:
        dict['url'] = "Pending"
    list_for_df.append(dict)
list_for_df

### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [ ]:
search_list = driver.find_elements_by_class_name('searchfieldtitle')
list_for_df = []

for result in search_list:
    dict = {}

    dict['name'] = result.find_elements_by_tag_name('td')[1].text
    try:
        dict['url'] = result.find_elements_by_tag_name('td')[2].find_element_by_tag_name('a').get_attribute('href')
    except:
        dict['url'] = "Pending"
    list_for_df.append(dict)
    
next_button = driver.find_element_by_partial_link_text('Next')
driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
next_button.click()

### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [12]:
while True:
    search_list = driver.find_elements_by_class_name('searchfieldtitle')
    list_for_df = []

    for result in search_list:
        dict = {}

        dict['name'] = result.find_elements_by_tag_name('td')[1].text
        try:
            dict['url'] = result.find_elements_by_tag_name('td')[2].find_element_by_tag_name('a').get_attribute('href')
        except:
            dict['url'] = "Pending"
        list_for_df.append(dict)

    next_button = driver.find_element_by_partial_link_text('Next')
    driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
    next_button.click()
    print("Scraping a new page")

Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.4 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [14]:
try:
    list_for_df = []
    while True:
        search_list = driver.find_elements_by_class_name('searchfieldtitle')

        for result in search_list:
            dict = {}

            dict['name'] = result.find_elements_by_tag_name('td')[1].text
            try:
                dict['url'] = result.find_elements_by_tag_name('td')[2].find_element_by_tag_name('a').get_attribute('href')
            except:
                dict['url'] = "Pending"
            list_for_df.append(dict)

        next_button = driver.find_element_by_partial_link_text('Next')
        driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
        next_button.click()
        print("Scraping a new page")
except:
    pass

Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page


In [15]:
list_for_df

[{'name': 'VAPE IT STORE II',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=iukTxbewFmQ%3D'},
 {'name': 'VAPE IT STORE I',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=mhuGD1c%2BSlI%3D'},
 {'name': 'VAPEPAD THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=4gn7%2BpOikQo%3D'},
 {'name': 'VAPE FROG', 'url': 'Pending'},
 {'name': 'VAPE FROG', 'url': 'Pending'},
 {'name': 'VAPE LOFT THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ayHxAl3emr4%3D'},
 {'name': 'VAPE N CIGAR',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=%2F9jwyW3e7F4%3D'},
 {'name': 'VAPE DOJO',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=W58kd05sEqg%3D'},
 {'name': 'VAPE HAVEN', 'url': 'Pending'},
 {'name': 'VAPORS LOUNGE', 'url': 'Pending'},
 {'name': 'VAPORFI SOUTHSIDE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=c4%2FPAcsaxFc%3D'},
 {'na

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [16]:
df = pd.DataFrame(list_for_df)

In [17]:
df.to_csv("vape-shops-basic.csv", index=False)

# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [18]:
df = pd.read_csv("vape-shops-basic.csv")
pd.set_option('display.max_colwidth', -1)
df.head()

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=iukTxbewFmQ%3D
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=mhuGD1c%2BSlI%3D
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=4gn7%2BpOikQo%3D
3,VAPE FROG,Pending
4,VAPE FROG,Pending


### Use Selenium to visit that page

In [ ]:
#driver.get(df.url[0])


### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [ ]:
rows = driver.find_elements_by_class_name('tablecelltext')

mailing_address = " ".join(rows[0].find_elements_by_tag_name('td')[0].text.split('\n')[-2:])
location_address = " ".join(rows[1].find_elements_by_tag_name('td')[0].text.split('\n')[-2:])
license_info = rows[0].find_elements_by_tag_name('td')[1].text
total_amt_paid = rows[-3].text.split(': ')[-1]
issued_by = " ".join(rows[-2].text.split('\n'))

first_dict = {
    'Mailing_address': mailing_address,
    'Location_address': location_address,
    'License_info': license_info,
    'Total_amt_paid': total_amt_paid,
    'Issued_by': issued_by
}

# return_to_list_button = driver.find_element_by_partial_link_text('Retur')
# return_to_list_button.click()
first_dict

In [ ]:
# testing cell
# rows = driver.find_elements_by_class_name('tablecelltext')
# rows[-2].text.split('\n')

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [19]:
driver.get(df.url[0])

rows = driver.find_elements_by_class_name('tablecelltext')

mailing_address = " ".join(rows[0].find_elements_by_tag_name('td')[0].text.split('\n')[-2:])
location_address = " ".join(rows[1].find_elements_by_tag_name('td')[0].text.split('\n')[-2:])
license_info = rows[0].find_elements_by_tag_name('td')[1].text
total_amt_paid = rows[-3].text.split(': ')[-1]
issued_by = " ".join(rows[-2].text.split('\n'))

first_dict = {
    'Mailing_address': mailing_address,
    'Location_address': location_address,
    'License_info': license_info,
    'Total_amt_paid': total_amt_paid,
    'Issued_by': issued_by
}
first_dict

{'Mailing_address': '1104 PLANTERS PLACE SALISBURY, MD 21804',
 'Location_address': '1015 S SALISBURY BLVD SALISBURY, MD 21801',
 'License_info': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury',
 'Total_amt_paid': '$ 41.00',
 'Issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT P.O. BOX 198 SALISBURY, MARYLAND 21803-0198'}

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [21]:
def get_details(row):
    driver.get(row['url'])

    rows = driver.find_elements_by_class_name('tablecelltext')

    mailing_address = " ".join(rows[0].find_elements_by_tag_name('td')[0].text.split('\n')[-2:])
    location_address = " ".join(rows[1].find_elements_by_tag_name('td')[0].text.split('\n')[-2:])
    license_info = rows[0].find_elements_by_tag_name('td')[1].text
    total_amt_paid = rows[-3].text.split(': ')[-1]
    issued_by = " ".join(rows[-2].text.split('\n'))
                   
    return pd.Series(first_dict)

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [23]:
driver = webdriver.Chrome()

# Take every row and send it to get_details, and combine with old data
new_df = df.apply(get_details, axis=1).join(df)
new_df.head()

IndexError: ('list index out of range', 'occurred at index 0')

## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.